# Analysis of new new algorithm

In [1]:
from IPython.display import display, HTML
from ipywidgets import interact, interactive
from py2neo import Node, Relationship, Graph
from scipy.linalg import toeplitz
from scipy.stats import binom
from tabulate import tabulate
from tqdm import tqdm_notebook as tqdm

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle as pk
import pymania as mn

{'Constantine': <class 'pymania.solvers.constantine.Constantine'>, 'MRP_F': <class 'pymania.solvers.mrp_symm_force.MRP_F'>, 'MRP_NF': <class 'pymania.solvers.mrp_symm_noforce.MRP_NF'>, 'Solver': <class 'pymania.solvers.base.Solver'>}


In [2]:
%matplotlib widget
graph = Graph(host="canopus.cc.gatech.edu",password='1234')
sub_train = [126426, 137431, 144125, 146735, 152427, 153227, 177140, 180533, 186545, 188145]
sub_test  = [206323, 227533, 248238, 360030, 361234, 362034, 368753, 401422, 413934, 453542]

run_id_dense = "Constantine_dense_v2"
run_id_sparse = "Constantine_sparse_v2"
run_id_vdense = "Constantine_vdense_v2"
run_id_vsparse = "Constantine_vsparse_v2"
run_id_all = [run_id_dense, run_id_sparse, run_id_vdense, run_id_vsparse]
run_label = {"Constantine_dense_v2":"Dense", "Constantine_sparse_v2":"Sparse", 
             "Constantine_vdense_v2":"Very Dense", "Constantine_vsparse_v2":"Very Sparse"}

all_subjects = sub_train + sub_test
n_train, n_test = len(sub_train), len(sub_test)

In [3]:
def joini(lst):
    return ', '.join(str(x) for x in lst)

# Run Mania on all Subjects

In [4]:
def get_network(subjects, run_type):
    network = mn.create_project('Constantine', run_type)
    for roi in ['L'+str(xx) for xx in range(1,181)]:
        network.add_roi(roi)
    for subject in subjects:
        network.add_subject(subject)
    network.load()
    network.run()
    return network

In [5]:
dense_network = get_network(sub_train, 'dense_temp')
sparse_network = get_network(sub_train, 'sparse_temp')
vdense_network = get_network(sub_train, 'vdense_temp')
vsparse_network = get_network(sub_train, 'vsparse_temp')

In [6]:
network = {'dense':dense_network, 'sparse':sparse_network, 'vdense':vdense_network, 'vsparse':vsparse_network}
suffixes = ['vsparse', 'sparse', 'dense', 'vdense']

# Network Density and it's Consistency across subjects
First, we try to see the total network density over all subjects and the consistency across each subject

In [7]:
query = f"MATCH (n:ROI)-[r:MANIA2]->(m:ROI) WHERE n.name STARTS WITH 'L' AND r.is_connected=TRUE AND "
query += f"r.run_id IN {run_id_all} WITH r.run_id as run_id, COUNT(*) AS connections, COUNT(DISTINCT r.SUBJECT) AS num_sub, AVG(r.threshold2) as mean2 "
query += "RETURN run_id, num_sub, connections*100.0/(32220.0*num_sub) as density, AVG(mean2) as threshold"
densities = graph.run(query).to_data_frame()

In [8]:
html = '<table><tr><th>Method</th><th># Subjects</th><th>Avg. Threshold</th><th>Density</th></tr>'
for index, row in densities.iterrows():
    html += f"<tr><td>{run_label[row['run_id']]}</td><td>{row['num_sub']}</td>"
    html += "<td>%7.2f</td><td>%5.2f%%</td></tr>" % (row['threshold'], row['density'])
html += '</table>'
display(HTML(html))

Method,# Subjects,Avg. Threshold,Density
Very Sparse,20,3228.15,3.10%
Dense,20,44.23,23.82%
Very Dense,20,85.35,27.14%
Sparse,20,516.05,11.72%


In [9]:
def get_subject_thresholds(run_id, subjects):
    query = f"MATCH (n:ROI)-[r:MANIA2]->(m:ROI) WHERE n.name STARTS WITH 'L' AND r.is_connected=TRUE AND "
    query += f" r.SUBJECT IN {subjects} AND r.run_id='{run_id}'  "
    query += "WITH r.SUBJECT as subject, COUNT(*) AS connections, AVG(r.threshold2) as threshold "
    query += "RETURN subject, connections, connections*100.0/32220.0 as percentage, threshold ORDER BY percentage"
    return graph.run(query).to_data_frame()

In [10]:
dense_thresholds = get_subject_thresholds(run_id_dense, sub_train)
sparse_thresholds = get_subject_thresholds(run_id_sparse, sub_train)
vdense_thresholds = get_subject_thresholds(run_id_vdense, sub_train)
vsparse_thresholds = get_subject_thresholds(run_id_vsparse, sub_train)

In [11]:
threshold1 = dense_thresholds.merge(sparse_thresholds, how='inner', on=['subject'], suffixes=('_dense', '_sparse'))
threshold2 = vdense_thresholds.merge(vsparse_thresholds, how='inner', on=['subject'], suffixes=('_vdense', '_vsparse'))
thresholds = threshold1.merge(threshold2, how='inner', on=['subject'])

In [12]:
html = '<table><tr><th rowspan="2" align="center" >Subject</th><th colspan="3">Very Sparse</th>'
html += '<th colspan="3">Sparse</th><th colspan="3">Dense</th><th colspan="3">Very Dense</th></tr><tr>'
for i in range(4):
    html += '<th>Density</th><th>Threshold</th><th>NAR</th>'
html += '</tr>'
for index, row in thresholds.iterrows():
    html += "<tr><td>%d</td>" % row['subject']
    for suffix in suffixes:
        html += "<td>%5.2f%%</td><td>%d</td>" % (row['percentage_'+suffix], row['threshold_'+suffix])
        html += '<td>%5.3f</td>' % mn.utils.NAR(network[suffix](row['subject']).mania2_network)
    html += "</tr>"
display(HTML(html))

# Consistency of Adjacent Nodes

In [13]:
query = f"MATCH (n:ROI)-[r:MANIA2]->(m:ROI) WHERE n.name STARTS WITH 'L' AND r.correction_type='Adjacent' "
query += f"AND r.SUBJECT IN {sub_train} AND r.run_id IN {run_id_all} RETURN r.run_id AS run_id, "
query += "r.SUBJECT as subject, r.is_connected AS is_connected, COUNT(*) AS count"
adjacency_frame = graph.run(query).to_data_frame()

In [14]:
adjacency_counts = dict()
for index, row in adjacency_frame.iterrows():
    subject, run_id = row['subject'], row['run_id']
    if subject not in adjacency_counts:
        adjacency_counts[subject] = dict()
    if run_id not in adjacency_counts[subject]:
        adjacency_counts[subject][run_id] = {True:0, False:0, 'Total':0}
    adjacency_counts[subject][run_id][row['is_connected']] += row['count']

In [15]:
html = '<table><tr><th rowspan="2" align="center" >Subject</th><th colspan="2">Very Sparse</th>'
html += '<th colspan="2">Sparse</th><th colspan="2">Dense</th><th colspan="2">Very Dense</th></tr><tr>'
for i in range(4):
    html += '<th>% of Adjancent Nodes</th><th>% Adj. Connected</th>'
html += '</tr>'
for subject, values in adjacency_counts.items():
    html += f'<tr><td>{subject}</td>'
    for suffix in suffixes:
        counts = values[f'Constantine_{suffix}_v2']
        pct_adj = (counts[True] + counts[False]) / 322.20
        pct_con = counts[True] * 100. / (counts[True] + counts[False])
        html += '<td>%5.2f%%</td><td>%5.2f%%</td>' % (pct_adj, pct_con)
    html += '</tr>'
html += '</table>'
display(HTML(html))

In [16]:
query = f"MATCH (n:ROI)-[r:MANIA2]->(m:ROI) WHERE n.name STARTS WITH 'L' AND r.correction_type='Adjacent' "
query += f"AND r.SUBJECT IN {sub_train} AND r.run_id IN {run_id_all} "
query += "WITH r.run_id AS run_id, n.name AS source, m.name AS target, COUNT(*) AS num_subjects "
query += " RETURN run_id, num_subjects, COUNT(*) AS num_links"
adjacency_count = graph.run(query).to_data_frame()

In [17]:
adjacency_counts = dict()
for index, row in adjacency_count.iterrows():
    run_id = str(row['run_id'].split('_')[1])
    if run_id not in adjacency_counts:
        adjacency_counts[run_id] = np.zeros(len(sub_train)+1, dtype=int)
    adjacency_counts[run_id][row['num_subjects']] = row['num_links']

In [18]:
plt.figure()
x_axis = np.arange(0, len(sub_train)+1)
plt.bar(x_axis-0.3, adjacency_counts['vsparse'], width=0.2, label='Very Sparse')
plt.bar(x_axis-0.1, adjacency_counts['sparse'], width=0.2, label='Sparse')
plt.bar(x_axis+0.1, adjacency_counts['dense'], width=0.2, label='Dense')
plt.bar(x_axis+0.3, adjacency_counts['vdense'], width=0.2, label='Very Dense')
plt.legend()
plt.title('Consistency of Adjacency of Links')
plt.xlabel('# Subjects in which the Link is Adjacent')
plt.ylabel('# of such Links')
plt.show()

FigureCanvasNbAgg()

In [19]:
query = f"MATCH (n:ROI)-[r:MANIA2]->(m:ROI) WHERE n.name STARTS WITH 'L' AND r.correction_type='Adjacent' "
query += f"AND r.SUBJECT IN {sub_train} AND r.run_id IN {run_id_all} AND r.is_connected=TRUE "
query += "WITH r.run_id AS run_id, n.name AS source, m.name AS target, COUNT(*) AS num_subjects "
query += " RETURN run_id, num_subjects, COUNT(*) AS num_links"
adj_connected_count = graph.run(query).to_data_frame()

In [20]:
adj_connected_counts = dict()
for index, row in adj_connected_count.iterrows():
    run_id = str(row['run_id'].split('_')[1])
    if run_id not in adj_connected_counts:
        adj_connected_counts[run_id] = np.zeros(len(sub_train)+1, dtype=int)
    adj_connected_counts[run_id][row['num_subjects']] = row['num_links']

In [21]:
plt.figure()
x_axis = np.arange(0, len(sub_train)+1)
plt.bar(x_axis-0.3, adj_connected_counts['vsparse'], width=0.2, label='Very Sparse')
plt.bar(x_axis-0.1, adj_connected_counts['sparse'], width=0.2, label='Sparse')
plt.bar(x_axis+0.1, adj_connected_counts['dense'], width=0.2, label='Dense')
plt.bar(x_axis+0.3, adj_connected_counts['vdense'], width=0.2, label='Very Dense')
plt.legend()
plt.title('Consistency of Connected Adjacent Links')
plt.xlabel('# Subjects in which the Link is Adjacent and Connected')
plt.ylabel('# of such Links')
plt.show()

FigureCanvasNbAgg()

# Mania Plots

In [22]:
plot_manias_cache = dict()
def plot_manias(subject):
    subject = int(subject)
    if subject not in plot_manias_cache:
        dense_sub, sparse_sub = dense_network(subject), sparse_network(subject)
        vdense_sub, vsparse_sub = vdense_network(subject), vsparse_network(subject)
        _,den_1,nar_1,t_1 = mn.utils.networks.mania_on_mat(dense_sub.matrix1)
        _,den_d,nar_d,t_d = mn.utils.networks.mania_on_mat(dense_sub.matrix2)
        _,den_s,nar_s,t_s = mn.utils.networks.mania_on_mat(sparse_sub.matrix2)
        _,den_vd,nar_vd,t_vd = mn.utils.networks.mania_on_mat(vdense_sub.matrix2)
        _,den_vs,nar_vs,t_vs = mn.utils.networks.mania_on_mat(vsparse_sub.matrix2)
        threshold_1 = dense_sub.threshold1
        threshold_d, threshold_vd = dense_sub.threshold2, vdense_sub.threshold2
        threshold_s, threshold_vs = sparse_sub.threshold2, vsparse_sub.threshold2
        idx_1, idx_d, idx_s = np.argmin(nar_1), np.argmin(nar_d), np.argmin(nar_s)
        idx_vd, idx_vs = np.argmin(nar_vd), np.argmin(nar_vs)
        density_1, density_d, density_s = den_1[idx_1], den_d[idx_d], den_s[idx_s]
        density_vd, density_vs = den_vd[idx_vd], den_vs[idx_vs]
        nar_value_1, nar_value_d, nar_value_s = nar_1[idx_1], nar_d[idx_d], nar_s[idx_s]
        nar_value_vd, nar_value_vs = nar_vd[idx_vd], nar_vs[idx_vs]
        d = dict()
        d['den_1'], d['den_d'], d['den_s'], d['den_vd'], d['den_vs'] = den_1, den_d, den_s, den_vd, den_vs
        d['nar_1'], d['nar_d'], d['nar_s'], d['nar_vd'], d['nar_vs'] = nar_1, nar_d, nar_s, nar_vd, nar_vs
        d['density_1'], d['density_d'], d['density_s'], d['density_vd'], d['density_vs'] = density_1, density_d, density_s, density_vd, density_vs
        d['nar_value_1'], d['nar_value_d'], d['nar_value_s'], d['nar_value_vd'], d['nar_value_vs'] = nar_value_1, nar_value_d, nar_value_s, nar_value_vd, nar_value_vs
        d['t_1'], d['t_d'], d['t_s'], d['t_vd'], d['t_vs'] = t_1, t_d, t_s, t_vd, t_vs
        d['threshold_1'], d['threshold_d'], d['threshold_s'], d['threshold_vd'], d['threshold_vs'] = threshold_1, threshold_d, threshold_s, threshold_vd, threshold_vs
        plot_manias_cache[subject] = d
    else:
        d = plot_manias_cache[subject]
        den_1, den_d, den_s, den_vd, den_vs = d['den_1'], d['den_d'], d['den_s'], d['den_vd'], d['den_vs']
        nar_1, nar_d, nar_s, nar_vd, nar_vs = d['nar_1'], d['nar_d'], d['nar_s'], d['nar_vd'], d['nar_vs']
        density_1, density_d, density_s, density_vd, density_vs = d['density_1'], d['density_d'], d['density_s'], d['density_vd'], d['density_vs']
        nar_value_1, nar_value_d, nar_value_s, nar_value_vd, nar_value_vs = d['nar_value_1'], d['nar_value_d'], d['nar_value_s'], d['nar_value_vd'], d['nar_value_vs']
        t_1, t_d, t_s, t_vd, t_vs = d['t_1'], d['t_d'], d['t_s'], d['t_vd'], d['t_vs']
        threshold_1, threshold_d, threshold_s, threshold_vd, threshold_vs = d['threshold_1'], d['threshold_d'], d['threshold_s'], d['threshold_vd'], d['threshold_vs']
    
    fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(10,6))
    ax[0].plot(den_1,nar_1,'b-',lw=2,label='MANIA 1')
    ax[0].plot(den_vs,nar_vs,'c-',lw=2,label='Very Sparse')
    ax[0].plot(den_s,nar_s,'g-',lw=2,label='Sparse')
    ax[0].plot(den_d,nar_d,'r-',lw=2,label='Dense')
    ax[0].plot(den_vd,nar_vd,'m-',lw=2,label='Very Dense')
    ax[0].axvline(density_1, 0, 1, color='b', linestyle='dashed', lw=1)
    ax[0].axvline(density_d, 0, 1, color='r', linestyle='dashed', lw=1)
    ax[0].axvline(density_s, 0, 1, color='g', linestyle='dashed', lw=1)
    ax[0].axvline(density_vd, 0, 1, color='c', linestyle='dashed', lw=1)
    ax[0].axvline(density_vs, 0, 1, color='k', linestyle='dashed', lw=1)
    ax[0].axhline(nar_value_1, 0, 1, color='b', linestyle='dashed', lw=1)
    ax[0].axhline(nar_value_d, 0, 1, color='r', linestyle='dashed', lw=1)
    ax[0].axhline(nar_value_s, 0, 1, color='g', linestyle='dashed', lw=1)
    ax[0].axhline(nar_value_vd, 0, 1, color='m', linestyle='dashed', lw=1)
    ax[0].axhline(nar_value_vs, 0, 1, color='c', linestyle='dashed', lw=1)
    ax[0].set_xlabel('density')
    ax[0].set_ylabel('NAR')
    ax[1].plot(t_1,den_1,'b-',lw=2,label='MANIA 1')
    ax[1].plot(t_vs,den_vs,'c-',lw=2,label='Very Sparse')
    ax[1].plot(t_s,den_s,'g-',lw=2,label='Sparse')
    ax[1].plot(t_d,den_d,'r-',lw=2,label='Dense')
    ax[1].plot(t_vd,den_vd,'m-',lw=2,label='Very Dense')
    ax[1].set_xlabel('Threshold')
    ax[1].set_ylabel('Density')
    ax[1].axvline(threshold_1, color='b', linestyle='dashed', lw=1)
    ax[1].axvline(threshold_d, color='r', linestyle='dashed', lw=1)
    ax[1].axvline(threshold_s, color='g', linestyle='dashed', lw=1)
    ax[1].axvline(threshold_vd, color='m', linestyle='dashed', lw=1)
    ax[1].axvline(threshold_vs, color='c', linestyle='dashed', lw=1)
    ax[1].axhline(density_1, 0, 1, color='b', linestyle='dashed', lw=1)
    ax[1].axhline(density_d, 0, 1, color='r', linestyle='dashed', lw=1)
    ax[1].axhline(density_s, 0, 1, color='g', linestyle='dashed', lw=1)
    ax[1].axhline(density_vd, 0, 1, color='c', linestyle='dashed', lw=1)
    ax[1].axhline(density_vs, 0, 1, color='k', linestyle='dashed', lw=1)
    plt.suptitle(f'Subject: {subject}')
    plt.legend()
    plt.show()

In [23]:
interact(plot_manias, subject=[str(x) for x in sub_train])

interactive(children=(Dropdown(description='subject', options=('126426', '137431', '144125', '146735', '152427…

<function __main__.plot_manias(subject)>

# Correction Types

In [24]:
def get_type(corr_type, reg_type, sparse):
    if corr_type == 'Null':
        return 'Null'
    if corr_type == 'Adjacent':
        return 'Adjacent'
    if corr_type in ['Bad regressor', 'No Envelope No regress']:
        return 'No Correction'
    if corr_type == 'Regress' and reg_type == 'independent':
        return 'Independent'
    if sparse:
        if reg_type in ['direction1', 'direction2', 'poolAll', 'poolEnvelopes']:
            return 'No Correction'
    else:
        if reg_type == 'direction1' or reg_type == 'direction2':
            return 'one direction'
        if reg_type == 'poolAll' or reg_type == 'poolEnvelopes':
            return reg_type
    if corr_type == 'No Correction':
        return 'No Correction'
    print('Error while evaluating: ', corr_type, reg_type)

In [25]:
def get_filter(algo_type):
    assert (algo_type in types)
    if algo_type in ['Null', 'Adjacent'] :
        return f"r.correction_type='{algo_type}'"
    if algo_type == 'No Correction':
        return "r.correction_type IN ['Bad regressor', 'No Envelope No regress', 'No Correction']"
    if algo_type == 'Independent':
        return "(r.correction_type='Regress' AND r.regressor_type='independent')"
    if algo_type == 'one direction':
        return "(r.correction_type IN ['No Envelope but regress', 'Regress'] AND r.regressor_type IN ['direction1', 'direction2'])"
    if algo_type == 'poolAll':
        return "(r.correction_type IN ['No Envelope but regress', 'Regress'] AND r.regressor_type = 'poolAll')"
    string = "("
    first = True
    for (corr_type, reg_type) in types[algo_type]:
        if not first:
            string += "OR "
        else:
            first = False
        string += f"(r.correction_type='{corr_type}' AND r.regressor_type='{reg_type}') "
    string += ")"
    return string

In [26]:
types = {'Null':set(), 'Adjacent':set(), 'No Correction':set(), 'Independent':set(), 'one direction':set(), 'poolAll':set()}
def get_counts_by_type(run_id, subjects):
    query = f"MATCH (n:ROI)-[r:MANIA2]->(m:ROI) WHERE n.name STARTS WITH 'L' AND r.SUBJECT IN [{joini(subjects)}] "
    query += f"AND r.is_connected=TRUE AND r.run_id='{run_id}' "
    query += "RETURN r.correction_type AS correction_type, r.regressor_type AS regressor_type, COUNT(*) AS counts"
    counts_true = graph.run(query).to_ndarray()
    query = f"MATCH (n:ROI)-[r:MANIA2]->(m:ROI) WHERE n.name STARTS WITH 'L' AND r.SUBJECT IN [{joini(subjects)}] "
    query += f"AND r.is_connected=FALSE AND r.run_id='{run_id}' "
    query += f"RETURN r.correction_type AS correction_type, r.regressor_type AS regressor_type, COUNT(*) AS counts"
    counts_false = graph.run(query).to_ndarray()
    counts = dict()
    for row in counts_true:
        algo_type = get_type(row[0], row[1], 'sparse' in run_id)
        types[algo_type].add((row[0], row[1]))
        if algo_type not in counts:
            counts[algo_type] = {True:0, False:0}
        counts[algo_type][True] += int(row[2])
    for row in counts_false:
        algo_type = get_type(row[0], row[1], 'sparse' in run_id)
        types[algo_type].add((row[0], row[1]))
        if algo_type not in counts:
            counts[algo_type] = {True:0, False:0}
        counts[algo_type][False] += int(row[2])
    for algo_type in counts:
        counts[algo_type]['total'] = counts[algo_type][True] + counts[algo_type][False]
    total = 0
    for algo_type in counts:
        total += counts[algo_type]['total']
    assert total == len(subjects)*32220
    for algo_type in counts:
        counts[algo_type]['fraction'] = counts[algo_type]['total']/float(total)
        counts_all = counts[algo_type][True] + counts[algo_type][False]
        counts[algo_type]['pct_connected'] = counts[algo_type][True]*100./counts_all
    return counts

In [27]:
by_type_sparse = get_counts_by_type(run_id_sparse, sub_train)
by_type_vdense = get_counts_by_type(run_id_vdense, sub_train)
by_type_dense = get_counts_by_type(run_id_dense, sub_train)
by_type_vsparse = get_counts_by_type(run_id_vsparse, sub_train)

In [112]:
html = '<table><tr><th rowspan="2">Type</th><th colspan="2">Very Sparse</th><th colspan="2">Sparse</th>'
html += '<th colspan="2">Dense</th><th colspan="2">Very Dense</th></tr><tr>'
for i in range(4):
    html += '<th>Fraction of Total</th><th>% Connected</th>'
html += '</tr>'
for algo_type in ['Adjacent', 'Independent', 'poolAll', 'one direction', 'No Correction', 'Null']:
    if algo_type in by_type_dense:
        dense = by_type_dense[algo_type]
    else:
        dense = {'fraction':0., 'pct_connected':0.}
    if algo_type in by_type_sparse:
        sparse = by_type_sparse[algo_type]
    else:
        sparse = {'fraction':0., 'pct_connected':0.}
    if algo_type in by_type_vdense:
        vdense = by_type_vdense[algo_type]
    else:
        dense = {'fraction':0., 'pct_connected':0.}
    if algo_type in by_type_vsparse:
        vsparse = by_type_vsparse[algo_type]
    else:
        sparse = {'fraction':0., 'pct_connected':0.}
    html += '<tr><td>%s</td>' % algo_type
    html += '<td>%5.2f%%</td><td>%5.2f%%</td>' % (vsparse['fraction']*100, vsparse['pct_connected'])
    html += '<td>%5.2f%%</td><td>%5.2f%%</td>' % (sparse['fraction']*100, sparse['pct_connected'])
    html += '<td>%5.2f%%</td><td>%5.2f%%</td>' % (dense['fraction']*100, dense['pct_connected'])
    html += '<td>%5.2f%%</td><td>%5.2f%%</td></tr>' % (vdense['fraction']*100, vdense['pct_connected'])
html += '</table>'
display(HTML(html))

# Sample Links from Different types

In [ ]:
def get_sample_links(subjects, run_id, algo_type, num_links = 10):
    query = f"MATCH (n:ROI)-[r:MANIA2]->(m:ROI) WHERE n.name STARTS WITH 'L' AND r.run_id='{run_id}' "
    query += f"AND r.SUBJECT IN {subjects} AND {get_filter(algo_type)} "
    query += "WITH n.name AS source, m.name AS target, r.SUBJECT AS subject "
    query += f"RETURN subject, source, target LIMIT {num_links}"
    return graph.run(query).to_data_frame()

links_sparse = dict()
for algo_type in types:
    links_sparse[algo_type] = get_sample_links(sub_train, run_id_sparse, algo_type, 4)

In [ ]:
for algo_type in types:
    display(HTML(f'<h2>{algo_type}</h2>'))
    for idx, row in links_sparse[algo_type].iterrows():
        pair = dense_network(row['subject'])(row['source'], row['target'],pair=True)
        print(row)
        print(pair.st1.regressor.kind, pair.st2.regressor.kind)
        pair.plot()
        plt.show()

In [ ]:
def get_sample_links(subjects, run_id, algo_type, num_links = 10):
    query = f"MATCH (n:ROI)-[r:MANIA2]->(m:ROI) WHERE n.name STARTS WITH 'L' AND r.run_id='{run_id}' "
    query += f"AND r.SUBJECT IN {subjects} AND {get_filter(algo_type)} "
    query += "WITH n.name AS source, m.name AS target, r.SUBJECT AS subject "
    query += f"RETURN subject, source, target LIMIT {num_links}"
    return graph.run(query).to_data_frame()

## Dense version

In [ ]:
links_dense = dict()
for algo_type in types:
    display(HTML(f'<h3>{algo_type}</h3>'))
    links_dense[algo_type] = get_sample_links(sub_train, run_id_dense, algo_type, 4)
    for idx, row in links[algo_type].iterrows():
        pair = dense_network(row['subject'])(row['source'], row['target'],pair=True)
        print(pair.st1.regressor.kind, pair.st2.regressor.kind)
        pair.plot()
        plt.show()

## Sparse Version

In [ ]:
links_sparse = dict()
for algo_type in types:
    display(HTML(f'<h3>{algo_type}</h3>'))
    links_sparse[algo_type] = get_sample_links(sub_train, run_id_sparse, algo_type, 4)
    for idx, row in links_sparse[algo_type].iterrows():
        pair = sparse_network(row['subject'])(row['source'], row['target'],pair=True)
        print(pair.st1.regressor.kind, pair.st2.regressor.kind)
        pair.plot()
        plt.show()